Source: https://www.analyticsvidhya.com/blog/2021/12/fine-tune-bert-model-for-sentiment-analysis-in-google-colab/

https://huggingface.co/docs/transformers/training

In [1]:
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews',
          split = (tfds.Split.TRAIN, tfds.Split.TEST),
          as_supervised=True,
          with_info=True)

/Users/michaelcheng/miniforge3/envs/fyp-test-keybert/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-14 21:26:18.845314: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-10-14 21:26:18.845334: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-10-14 21:26:18.845337: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-10-14 21:26:18.845377: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-14 21:26:18.845397: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/devi

In [2]:
# convert to string
sentences_train = []
labels_train = []

sentences_test = []
labels_test = []

for sentence, label in ds_train:
    sentences_train.append(str(sentence.numpy()))
    labels_train.append(str(label.numpy()))

for sentence, label in ds_test:
    sentences_test.append(str(sentence.numpy()))
    labels_test.append(str(label.numpy()))

labels_train = list(map(int, labels_train))
labels_test = list(map(int, labels_test))

In [3]:
print(sentences_train[0])
print(type(sentences_train[0]))
print()
print(labels_train[0])
print(type(labels_train[0]))


b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
<class 'str'>

0
<class 'int'>


In [11]:
len(sentences_train)

25000

pytorch

In [4]:
# from above, we built a training dict and testing dict
# with format [{'label': 0, 'text': 'xxx'}, {'label': 0, 'text': 'xxx'}, {'label': 0, 'text': 'xxx'}, ...]

dataset_train = [
    {
        'label':l,
        'text': t
    }
    for t, l in zip(sentences_train, labels_train)
]

dataset_test = [
    {
        'label':l,
        'text': t
    }
    for t, l in zip(sentences_test, labels_test)
]

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


dataset_train_tokenized = dataset_train.map(tokenize_function, batched=True)
dataset_test_tokenized = dataset_test.map(tokenize_function, batched=True)

AttributeError: 'list' object has no attribute 'map'

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
)

In [12]:
trainer.train()

/Users/michaelcheng/miniforge3/envs/fyp-test-keybert/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/9375 [00:00<?, ?it/s]

ValueError: You have to specify either input_ids or inputs_embeds

tf

Next, load a tokenizer and tokenize the data as NumPy arrays. Note that the labels are already a list of 0 and 1s, so we can just convert that directly to a NumPy array without tokenization!

In [1]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer takes a list of strings
tokenized_data = tokenizer(sentences_train, return_tensors="np", padding=True, max_length=512, truncation=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

# labels = np.array(labels_train)  # Label is already an array of 0 and 1
labels = np.array(labels_train)

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'sentences_train' is not defined

In [16]:
list(tokenized_data.keys())

['input_ids', 'token_type_ids', 'attention_mask']

In [18]:
tokenized_data['input_ids'].shape

(25000, 512)

In [19]:
from transformers import TFAutoModelForSequenceClassification
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!

history = model.fit(tokenized_data, labels)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-10-14 21:41:37.282155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  4/782 [..............................] - ETA: 3:22:26 - loss: 0.6951